Election de 2022 premier tour

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['.DS_Store',
 'enums',
 'utils',
 'models',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'labo2.ipynb',
 '.git',
 'main.py',
 'elections']

#### Importer la database

In [2]:
from db import SessionLocal
from models import Departement

session = SessionLocal()



Import des datasets.

In [44]:
df = pd.read_excel('./elections/presidentielles-2012-1.xlsx')

Visualise toutes les colones

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1,Sexe.2,Nom.2,Prénom.2,Voix.2,% Voix/Ins.2,% Voix/Exp.2,Sexe.3,Nom.3,Prénom.3,Voix.3,% Voix/Ins.3,% Voix/Exp.3,Sexe.4,Nom.4,Prénom.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Sexe.5,Nom.5,Prénom.5,Voix.5,% Voix/Ins.5,% Voix/Exp.5,Sexe.6,Nom.6,Prénom.6,Voix.6,% Voix/Ins.6,% Voix/Exp.6,Sexe.7,Nom.7,Prénom.7,Voix.7,% Voix/Ins.7,% Voix/Exp.7,Sexe.8,Nom.8,Prénom.8,Voix.8,% Voix/Ins.8,% Voix/Exp.8,Sexe.9,Nom.9,Prénom.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9,Sexe.10,Nom.10,Prénom.10,Voix.10,% Voix/Ins.10,% Voix/Exp.10
0,1,Ain,415909,81530,19.60,334379,80.40,6342,1.52,1.90,2239,0.54,0.67,325798,78.33,97.43,F,LE PEN,Marine,81455,19.58,25.00,M,MACRON,Emmanuel,73692,17.72,22.62,M,FILLON,François,69804,16.78,21.43,M,MÉLENCHON,Jean-Luc,51736,12.44,15.88,M,DUPONT-AIGNAN,Nicolas,19788,4.76,6.07,M,HAMON,Benoît,16711,4.02,5.13,M,ASSELINEAU,François,3612,0.87,1.11,M,LASSALLE,Jean,3465,0.83,1.06,M,POUTOU,Philippe,3098,0.74,0.95,F,ARTHAUD,Nathalie,1842,0.44,0.57,M,CHEMINADE,Jacques,595,0.14,0.18
1,2,Aisne,375752,80183,21.34,295569,78.66,5047,1.34,1.71,2323,0.62,0.79,288199,76.70,97.51,F,LE PEN,Marine,102787,27.36,35.67,M,MACRON,Emmanuel,51693,13.76,17.94,M,MÉLENCHON,Jean-Luc,48959,13.03,16.99,M,FILLON,François,46985,12.50,16.30,M,DUPONT-AIGNAN,Nicolas,14652,3.90,5.08,M,HAMON,Benoît,12231,3.26,4.24,M,POUTOU,Philippe,3156,0.84,1.10,F,ARTHAUD,Nathalie,2764,0.74,0.96,M,LASSALLE,Jean,2265,0.60,0.79,M,ASSELINEAU,François,2171,0.58,0.75,M,CHEMINADE,Jacques,536,0.14,0.19
2,3,Allier,253946,54357,21.40,199589,78.60,4240,1.67,2.12,2556,1.01,1.28,192793,75.92,96.60,M,MACRON,Emmanuel,45744,18.01,23.73,F,LE PEN,Marine,43071,16.96,22.34,M,MÉLENCHON,Jean-Luc,38324,15.09,19.88,M,FILLON,François,36499,14.37,18.93,M,HAMON,Benoît,10639,4.19,5.52,M,DUPONT-AIGNAN,Nicolas,9819,3.87,5.09,M,LASSALLE,Jean,2988,1.18,1.55,M,POUTOU,Philippe,2328,0.92,1.21,F,ARTHAUD,Nathalie,1543,0.61,0.80,M,ASSELINEAU,François,1483,0.58,0.77,M,CHEMINADE,Jacques,355,0.14,0.18
3,4,Alpes-de-Haute-Provence,126578,24323,19.22,102255,80.78,1806,1.43,1.77,736,0.58,0.72,99713,78.78,97.51,F,LE PEN,Marine,24463,19.33,24.53,M,MÉLENCHON,Jean-Luc,22448,17.73,22.51,M,MACRON,Emmanuel,19960,15.77,20.02,M,FILLON,François,18442,14.57,18.50,M,HAMON,Benoît,4983,3.94,5.00,M,DUPONT-AIGNAN,Nicolas,4860,3.84,4.87,M,LASSALLE,Jean,1721,1.36,1.73,M,POUTOU,Philippe,1178,0.93,1.18,M,ASSELINEAU,François,932,0.74,0.93,F,ARTHAUD,Nathalie,521,0.41,0.52,M,CHEMINADE,Jacques,205,0.16,0.21
4,5,Hautes-Alpes,109907,20821,18.94,89086,81.06,1541,1.40,1.73,618,0.56,0.69,86927,79.09,97.58,M,MACRON,Emmanuel,18948,17.24,21.80,M,MÉLENCHON,Jean-Luc,18796,17.10,21.62,F,LE PEN,Marine,18474,16.81,21.25,M,FILLON,François,16645,15.14,19.15,M,HAMON,Benoît,5109,4.65,5.88,M,DUPONT-AIGNAN,Nicolas,4938,4.49,5.68,M,LASSALLE,Jean,1609,1.46,1.85,M,POUTOU,Philippe,1049,0.95,1.21,M,ASSELINEAU,François,783,0.71,0.90,F,ARTHAUD,Nathalie,411,0.37,0.47,M,CHEMINADE,Jacques,165,0.15,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,ZN,Nouvelle-Calédonie,189483,98257,51.86,91226,48.14,2969,1.57,3.25,1311,0.69,1.44,86946,45.89,95.31,M,FILLON,François,27065,14.28,31.13,F,LE PEN,Marine,25290,13.35,29.09,M,MACRON,Emmanuel,11089,5.85,12.75,M,HAMON,Benoît,8125,4.29,9.34,M,MÉLENCHON,Jean-Luc,7703,4.07,8.86,M,DUPONT-AIGNAN,Nicolas,2521,1.33,2.90,M,ASSELINEAU,François,2098,1.11,2.41,M,POUTOU,Philippe,1284,0.68,1.48,F,ARTHAUD,Nathalie,836,0.44,0.96,M,LASSALLE,Jean,695,0.37,0.80,M,CHEMINADE,Jacques,240,0.13,0.28
102,ZP,Polynés

In [5]:
df.shape

(106, 82)

In [45]:
df.insert(2,'etat_saisie','complet')

Recuperer les information généraux sur les élections départemental

In [53]:
df_infos_general = df.iloc[:, :1100]
df_infos_general.head()

,Code du département,Libellé du département,etat_saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1,Sexe.2,Nom.2,Prénom.2,Voix.2,% Voix/Ins.2,% Voix/Exp.2,Sexe.3,Nom.3,Prénom.3,Voix.3,% Voix/Ins.3,% Voix/Exp.3,Sexe.4,Nom.4,Prénom.4,Voix.4,% Voix/Ins.4,% Voix/Exp.4,Sexe.5,Nom.5,Prénom.5,Voix.5,% Voix/Ins.5,% Voix/Exp.5,Sexe.6,Nom.6,Prénom.6,Voix.6,% Voix/Ins.6,% Voix/Exp.6,Sexe.7,Nom.7,Prénom.7,Voix.7,% Voix/Ins.7,% Voix/Exp.7,Sexe.8,Nom.8,Prénom.8,Voix.8,% Voix/Ins.8,% Voix/Exp.8,Sexe.9,Nom.9,Prénom.9,Voix.9,% Voix/Ins.9,% Voix/Exp.9
0,1,AIN,complet,393808,65996,16.76,327812,83.24,6453,1.64,1.97,321359,81.60,98.03,F,JOLY,Eva,7268,1.85,2.26,F,LE PEN,Marine,66540,16.90,20.71,M,SARKOZY,Nicolas,97722,24.81,30.41,M,MÉLENCHON,Jean-Luc,30898,7.85,9.61,M,POUTOU,Philippe,3323,0.84,1.03,F,ARTHAUD,Nathalie,1794,0.46,0.56,M,CHEMINADE,Jacques,860,0.22,0.27,M,BAYROU,François,32650,8.29,10.16,M,DUPONT-AIGNAN,Nicolas,7208,1.83,2.24,M,HOLLANDE,François,73096,18.56,22.75
1,2,AISNE,complet,376068,72928,19.39,303140,80.61,5196,1.38,1.71,297944,79.23,98.29,F,JOLY,Eva,3455,0.92,1.16,F,LE PEN,Marine,78452,20.86,26.33,M,SARKOZY,Nicolas,72090,19.17,24.20,M,MÉLENCHON,Jean-Luc,30360,8.07,10.19,M,POUTOU,Philippe,3860,1.03,1.30,F,ARTHAUD,Nathalie,2490,0.66,0.84,M,CHEMINADE,Jacques,738,0.20,0.25,M,BAYROU,François,19895,5.29,6.68,M,DUPONT-AIGNAN,Nicolas,5853,1.56,1.96,M,HOLLANDE,François,80751,21.47,27.10
2,3,ALLIER,complet,256275,45266,17.66,211009,82.34,5059,1.97,2.40,205950,80.36,97.60,F,JOLY,Eva,3232,1.26,1.57,F,LE PEN,Marine,37736,14.72,18.32,M,SARKOZY,Nicolas,49477,19.31,24.02,M,MÉLENCHON,Jean-Luc,27969,10.91,13.58,M,POUTOU,Philippe,2584,1.01,1.25,F,ARTHAUD,Nathalie,1482,0.58,0.72,M,CHEMINADE,Jacques,457,0.18,0.22,M,BAYROU,François,17814,6.95,8.65,M,DUPONT-AIGNAN,Nicolas,4068,1.59,1.98,M,HOLLANDE,François,61131,23.85,29.68
3,4,ALPES DE HAUTE PROVENCE,complet,123933,21034,16.97,102899,83.03,2111,1.70,2.05,100788,81.32,97.95,F,JOLY,Eva,2933,2.37,2.91,F,LE PEN,Marine,20875,16.84,20.71,M,SARKOZY,Nicolas,25668,20.71,25.47,M,MÉLENCHON,Jean-Luc,15269,12.32,15.15,M,POUTOU,Philippe,1394,1.12,1.38,F,ARTHAUD,Nathalie,487,0.39,0.48,M,CHEMINADE,Jacques,283,0.23,0.28,M,BAYROU,François,7483,6.04,7.42,M,DUPONT-AIGNAN,Nicolas,1845,1.49,1.83,M,HOLLANDE,François,24551,19.81,24.36
4,5,HAUTES ALPES,complet,106865,18246,17.07,88619,82.93,1842,1.72,2.08,86777,81.20,97.92,F,JOLY,Eva,3147,2.94,3.63,F,LE PEN,Marine,15359,14.37,17.70,M,SARKOZY,Nicolas,22655,21.20,26.11,M,MÉLENCHON,Jean-Luc,12175,11.39,14.03,M,POUTOU,Philippe,1152,1.08,1.33,F,ARTHAUD,Nathalie,488,0.46,0.56,M,CHEMINADE,Jacques,212,0.20,0.24,M,BAYROU,François,8559,8.01,9.86,M,DUPONT-AIGNAN,Nicolas,1782,1.67,2.05,M,HOLLANDE,François,21248,19.88,24.49


Renommer les colonnes des information généraux sur les élections départemental

In [52]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "etat_saisie", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs",
    "pct_blancs_inscrits", "pct_blancs_votants", "nb_nuls",
    "pct_nuls_inscrits", "pct_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

ValueError: Length mismatch: Expected axis has 14 elements, new values have 17 elements

## Analyse exploratoire des données

Elle sert à vérifier que les données sont complètes et cohérentes, à identifier ce qui est réellement informatif (variables utiles) et ce qui est redondant ou inutile, à détecter d’éventuelles anomalies ou valeurs aberrantes, et à appréhender la structure globale des données (volumes, proportions, variabilité entre départements).

#### Structure globale du DataFrame

Permet de repérer les colonnes vides, les types incorrects, et d’avoir une première idée de la taille du jeu de données.

In [ ]:
df_infos_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code_dept              106 non-null    object 
 1   nom_dept               106 non-null    object 
 2   etat_saisie            106 non-null    object 
 3   nb_inscrits            106 non-null    int64  
 4   nb_abstentions         106 non-null    int64  
 5   pct_abstentions        106 non-null    float64
 6   nb_votants             106 non-null    int64  
 7   pct_votants            106 non-null    float64
 8   nb_blancs              106 non-null    int64  
 9   pct_blancs_inscrits    106 non-null    float64
 10  pct_blancs_votants     106 non-null    float64
 11  nb_nuls                106 non-null    int64  
 12  pct_nuls_inscrits      106 non-null    float64
 13  pct_nuls_votants       106 non-null    float64
 14  nb_exprimes            106 non-null    int64  
 15  pct_ex

#### Analyser les valeurs manquantes

Analyser les valeurs manquantes est essentiel car leur présence peut fausser l’analyse statistique et les modèles de prédiction : une valeur absente peut modifier les moyennes, les pourcentages ou les corrélations, introduire des biais et provoquer des erreurs ou de mauvaises performances lors de l’entraînement des modèles de machine learning.

In [ ]:
df_infos_general.isnull().sum()

#### Analyse descriptive statistique sur les colonnes numériques

Permet de comprendre la distribution, détecter outliers, différences de taille entre départements, etc.

Colonnes numériques pour analyse

In [ ]:
colonnes_numeriques = df_infos_general.select_dtypes(include=["int64", "float64"]).columns
df_num = df_infos_general[colonnes_numeriques]

print(df_num.describe())

In [ ]:
# 5a. Catégorielles restantes (ici code_dept)
for col in df_infos_general.select_dtypes(include=["object"]).columns:
    counts = df_infos_general[col].value_counts()
    sns.countplot(data=df_infos_general, x=col, order=counts.index)
    plt.title(f"Distribution de '{col}'")
    plt.xticks(rotation=45)
    plt.show()

#### Analyse descriptive des variables 

Permet de détecter les colonnes constantes ou redondantes

In [ ]:
df_infos_general.describe(include=["object"])

In [ ]:
# 5b. Numériques
for col in df_num.columns:
    sns.histplot(df_num[col], bins=10, kde=True)
    plt.title(f"Distribution de '{col}'")
    plt.show()

Recuperer que les ligne qui ont un état Complet

In [ ]:
df_infos_general = df_infos_general[df_infos_general["etat_saisie"] == "Complet"]

In [ ]:
msno.matrix(df)


Visualiser les valeurs manquantes

#### Analyser la distribution

Analyser la distribution des données permet de comprendre comment les valeurs se répartissent (concentration, dispersion, asymétrie) et de détecter d’éventuelles anomalies comme des valeurs aberrantes ou incohérentes.

Analyser les valeurs aberrantes

Analyse les boxes splots

In [ ]:
# 6️⃣ Boxplots pour détection des valeurs aberrantes
for col in df_num.columns:
    sns.boxplot(x=df_num[col])
    plt.title(f"Boxplot de '{col}'")
    plt.show()


Matrice de corrélation pour metre en evidance la muticolinéarité entre les variables numériques et un VIF pour mesure à quel point la variance d’un coefficient de régression est augmentée parce qu’une variable explicative est corrélée aux autres.

In [ ]:
# =========================
# 1️⃣ Matrice de corrélation
# =========================
corr_matrix = df_num.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matrice de corrélation des variables numériques")
plt.show()


In [ ]:
# =========================
# 2️⃣ VIF (Variance Inflation Factor)
# =========================

# Ajouter une constante pour le calcul du VIF
X = add_constant(df_num)

vif_data = pd.DataFrame()
vif_data["variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("VIF des variables numériques :")
print(vif_data)

Suppression des données redondantes, risque de multicolinéarité en ML

In [ ]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs", "nb_nuls"
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

In [ ]:
df_infos_general_base.shape

In [ ]:
df_departement = df_infos_general_base.iloc[:, :2]
df_departement.head()

In [ ]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

### Le test de ki2 pour verifier le lien entre deux variables qualitatif
### Le V2Cramer ou tupro pour mesurer l'intancite du lien

Entre deux varialbles quantitatif il faut faire des boxplot bivarier pour mesurer leurs relations

Test non parametrique entre une variable quantitatif et une variable quantitatif: krukal wallis

Infos sur les departements

Infos résultat des candidats par départements

### Partion

Partion de la base de donnée, tel que la base d'aprentissage = 0,7 et la base de test = 0,3

Tratification pour consever les proportions

In [ ]:
print(df)

In [ ]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 17:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran


# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

from db import SessionLocal
from models import Departement

session = SessionLocal()

df = pd.read_excel('./elections/presidentielles-2012-2.xlsx')

df.insert(2,'etat_saisie','complet')

In [50]:
# Exemple de configuration pour la classe ElectionDataFrame
election_config = {
    # Colonnes générales à conserver (infos départementales et électorales)
    "general_info_cols": [
        "code_dept", "nom_dept", "etat_saisie", "nb_inscrits",
        "nb_abstentions", "pct_abstentions", "nb_votants",
        "pct_votants", "nb_blancs", "pct_blancs_inscrits",
        "pct_blancs_votants", "nb_nuls", "pct_nuls_inscrits",
        "pct_nuls_votants", "nb_exprimes",
        "pct_exprimes_inscrits", "pct_exprimes_votants"
    ],

    # Colonnes numériques importantes pour les calculs
    "numeric_cols": ["nb_inscrits", "nb_abstentions", "nb_votants", "nb_blancs", "nb_nuls"],

    # Filtrage pour ne conserver que les départements complets
    "filter_column": "etat_saisie",
    "filter_value": "Complet",

    # Nombre de colonnes générales dans le dataframe
    "n_general_cols": 17,

    # Colonnes globales à conserver pour chaque candidat
    "cols_globales": ["Code du département", "Libellé du département"],

    # Colonnes par candidat dans le dataframe
    "candidate_columns": ["Sexe", "Nom", "Prenom", "Voix", "% Voix/Ins", "% Voix/Exp"],

    # Nombre de colonnes par candidat
    "n_cols_per_candidate": 6
}


In [ ]:
# 2012

election_config = {
    # =====================================================
    # Colonnes générales réelles du fichier
    # =====================================================
    "general_info_cols": [
        "code_dept",
        "nom_dept",
        "etat_saisie",
        "nb_inscrits",
        "nb_abstentions",
        "pct_abstentions",
        "nb_votants",
        "pct_votants",
        "nb_blancs",
        "nb_nuls",
        "pct_nuls_inscrits",
        "pct_nuls_votants",
        "nb_exprimes",
        "pct_exprimes_inscrits",
        "pct_exprimes_votants",
    ],

    # =====================================================
    # Colonnes numériques utiles (pour stats DB)
    # =====================================================
    "numeric_cols": ["nb_inscrits", "nb_abstentions", "nb_votants", "nb_blancs", "nb_nuls",
    ],

    # =====================================================
    # Filtrage des lignes valides
    # =====================================================
    "filter_column": "etat_saisie",
    "filter_value": "Complet",

    # =====================================================
    # Nombre réel de colonnes générales
    # (important pour découper les candidats)
    # =====================================================
    "n_general_cols": 14,

    # =====================================================
    # Colonnes communes répétées dans le df long
    # =====================================================
    "cols_globales": [
        "Code du département",
        "Libellé du département",
    ],

    # =====================================================
    # Colonnes par candidat (pattern répété)
    # =====================================================
    "candidate_columns": [
        "Sexe",
        "Nom",
        "Prenom",
        "Voix",
        "% Voix/Ins",
        "% Voix/Exp",
    ],

    # =====================================================
    # Taille d’un bloc candidat
    # =====================================================
    "n_cols_per_candidate": 6,
}


In [60]:
from utils.election_dataframe2 import ElectionDataFrame

# Créer l'objet ElectionDataFrame avec config
edf = ElectionDataFrame(df, election_config)

In [61]:
# Liste des candidats
edf.get_candidats()

,Sexe,Nom,Prenom
0,F,ARTHAUD,Nathalie
1,M,BAYROU,François
2,M,CHEMINADE,Jacques
3,M,DUPONT-AIGNAN,Nicolas
4,M,HOLLANDE,François
5,F,JOLY,Eva
6,F,LE PEN,Marine
7,M,MÉLENCHON,Jean-Luc
8,M,POUTOU,Philippe
9,M,SARKOZY,Nicolas


In [38]:
# Voix de Mélenchon par département
edf.get_voix_par_departement("LE PEN", "Marine")

,Code du département,Libellé du département,Voix


In [39]:
# Stats du département 59
edf.get_stats_departement(59)

,code_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs,nb_nuls
59,59,1792610,396412.0,1396198.0,87043.0,1309155.0


In [40]:
# Stats + voix pour Macron dans le 75
edf.get_stats_departement_candidat("59", "LE PEN", "Marine")

{'stats_departement':    code_dept  nb_inscrits  nb_abstentions  nb_votants  nb_blancs    nb_nuls
 59        59      1792610        396412.0   1396198.0    87043.0  1309155.0,
 'voix_candidat': Empty DataFrame
 Columns: [Voix]
 Index: []}

In [41]:
df_departement = edf.df_departement
print(df_departement.head())

  code_dept                 nom_dept
0        01                      AIN
1        02                    AISNE
2        03                   ALLIER
3        04  ALPES DE HAUTE PROVENCE
4        05             HAUTES ALPES


In [42]:
df_stat_elections = edf.df_stat_elections
print(df_stat_elections.head())

  code_dept  nb_inscrits  nb_abstentions  nb_votants  nb_blancs   nb_nuls
0        01       393866         67279.0    326587.0    19513.0  307074.0
1        02       376073         73997.0    302076.0    21056.0  281020.0
2        03       256211         45079.0    211132.0    14924.0  196208.0
3        04       123895         20314.0    103581.0     6639.0   96942.0
4        05       106875         17470.0     89405.0     6025.0   83380.0


In [43]:
df_candidat_resultat = edf.df_candidat_resultat
print(df_candidat_resultat.head())

  Code du département   Libellé du département      Sexe    Nom  Prenom Voix
0                  01                      AIN  131333.0  33.34   42.77    M
1                  02                    AISNE  147260.0  39.16   52.40    M
2                  03                   ALLIER  111615.0  43.56   56.89    M
3                  04  ALPES DE HAUTE PROVENCE   49498.0  39.95   51.06    M
4                  05             HAUTES ALPES   42435.0  39.71   50.89    M


In [62]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
election = importer.get_or_create_election(
    election_date=date(2012, 5, 6),
    type_election=TypeElection.PRESIDENTIELLE,
    tour=2
)

# =====================================================
# Départements
# =====================================================
df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")


2025-12-24 13:24:19,199 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-24 13:24:19,209 INFO sqlalchemy.engine.Engine SELECT elections.id AS elections_id, elections.date AS elections_date, elections.type_election AS elections_type_election, elections.tour AS elections_tour 
FROM elections 
WHERE elections.date = %(date_1)s AND elections.type_election = %(type_election_1)s AND elections.tour = %(tour_1)s 
 LIMIT %(param_1)s
2025-12-24 13:24:19,211 INFO sqlalchemy.engine.Engine [cached since 3573s ago] {'date_1': datetime.date(2012, 5, 6), 'type_election_1': 'PRESIDENTIELLE', 'tour_1': 2, 'param_1': 1}


2025-12-24 13:24:19,262 INFO sqlalchemy.engine.Engine SELECT departements.code_dept AS departements_code_dept, departements.nom_dept AS departements_nom_dept 
FROM departements 
WHERE departements.code_dept = %(pk_1)s
2025-12-24 13:24:19,263 INFO sqlalchemy.engine.Engine [cached since 3573s ago] {'pk_1': '01'}
2025-12-24 13:24:19,268 INFO sqlalchemy.engine.Engine SELECT departements.code_dept AS departements_code_dept, departements.nom_dept AS departements_nom_dept 
FROM departements 
WHERE departements.code_dept = %(pk_1)s
2025-12-24 13:24:19,269 INFO sqlalchemy.engine.Engine [cached since 3573s ago] {'pk_1': '02'}
2025-12-24 13:24:19,273 INFO sqlalchemy.engine.Engine SELECT departements.code_dept AS departements_code_dept, departements.nom_dept AS departements_nom_dept 
FROM departements 
WHERE departements.code_dept = %(pk_1)s
2025-12-24 13:24:19,274 INFO sqlalchemy.engine.Engine [cached since 3573s ago] {'pk_1': '03'}
2025-12-24 13:24:19,275 INFO sqlalchemy.engine.Engine SELECT dep

AttributeError: 'Series' object has no attribute 'nb_nuls'